<a href="https://colab.research.google.com/github/yunyoseob/github_test/blob/master/newyork_taxi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import io
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

train=pd.read_csv('train.csv', parse_dates=['pickup_datetime','dropoff_datetime'])
test=pd.read_csv('test.csv', parse_dates=['pickup_datetime'])
sample_submission=pd.read_csv('sample_submission.csv')

In [ ]:
df_train=pd.DataFrame(train)
df_test=pd.DataFrame(test) 
df_sample_submission=pd.DataFrame(sample_submission)

In [ ]:
fb_train=df_train.groupby(['id','pickup_datetime'])['trip_duration'].sum().reset_index()

In [ ]:
fb_train['ds']=fb_train['pickup_datetime']
fb_train['y']=fb_train['trip_duration']
fb_train.drop(['pickup_datetime','trip_duration'], axis=1, inplace=True)

In [ ]:
fb_test=df_test.drop(['vendor_id','passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'store_and_fwd_flag', 'ds'], axis=1)
fb_test['ds']=fb_test['pickup_datetime']
fb_test.drop(['pickup_datetime'], axis=1, inplace=True)

Prophet 모델 적용
--

In [ ]:
from fbprophet import Prophet

In [ ]:
model=Prophet()
model.fit(fb_train)
forecast=model.predict(fb_test)

In [ ]:
forecast=forecast.sort_values(by='ds', ascending=False)

In [ ]:
pred=forecast['yhat']

In [ ]:
submission_t1 = df_sample_submission['id']
submission_t1 = {"id":df_sample_submission["id"],"trip_duration":pred}
submission_t1 = pd.DataFrame(submission_t1)
submission_t1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 625134 entries, 0 to 625133
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             625134 non-null  object 
 1   trip_duration  625134 non-null  float64
dtypes: float64(1), object(1)
memory usage: 14.3+ MB


In [ ]:
submission_t1

,id,trip_duration
0,id3004672,881.212694
1,id3505355,881.178922
2,id1217141,880.861444
3,id2150126,880.701179
4,id1598245,880.354395
...,...,...
625129,id3008929,1049.889477
625130,id3700764,1049.844246
625131,id2568735,1049.810266
625132,id1384355,1049.776238


In [ ]:
submission_t1.to_csv('submission_t1.csv', index=False)